# step 2: classifiy adjectives in reviews as positive or negative

import librarys and prepare data set

In [1]:
# librarys
import pandas as pd
import nltk
import sklearn
import nltk.sentiment
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

import re
import pandas as pd
import nltk
import sklearn
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.tokenize import word_tokenize
import nltk.sentiment
import string
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import inflect
from textblob import TextBlob
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## define stopwords
sr = stopwords.words('english')
p = inflect.engine()
wnl = WordNetLemmatizer() 
table = str.maketrans('', '', string.punctuation)

Using TensorFlow backend.


In [2]:
# function to change text to words
def clean_review(text):
    #Change n't into not
    x = re.sub(r'n\'t',' not',text)
    #Change not word into not_word
    x = re.sub(r'not ','not_',x)
    #Split into words
    x = word_tokenize(x)
    #Remove punctuation
    x = [w.translate(table) if not re.match(r'not_.*', w) else w for w in x]
    #Change numbers into words
    x = [p.number_to_words(w) if w.isdigit() else w  for w in x ]
    #Remove non alphabetic
    x = [w for w in x if w.isalpha() or re.match(r'not_.*',w)]
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    return x

In [3]:
# load data
review=pd.read_csv("review_with_useful_features.csv")
review.head()

,business_id,stars,text
0,nsNONDHbV7Vudqh21uicqw,1.0,Was very excited for happy hour and heard grea...
1,nsNONDHbV7Vudqh21uicqw,2.0,Seems old and tired! I ate here frequently a...
2,nsNONDHbV7Vudqh21uicqw,1.0,"Poor service, had to sit there and ask my wife..."
3,nsNONDHbV7Vudqh21uicqw,1.0,Yesterday I took my husband to eat on his birt...
4,nsNONDHbV7Vudqh21uicqw,1.0,Second and last time. Sadly I had to wait for ...


In [4]:
# prepare data
review=review.drop(columns=['business_id'])
# we define stars from 1 to 3 as negative and stars from 4 to 5 as positive
review['stars']=np.where(review['stars'] >= 4, "positive", "negative")
review.head()

,stars,text
0,negative,Was very excited for happy hour and heard grea...
1,negative,Seems old and tired! I ate here frequently a...
2,negative,"Poor service, had to sit there and ask my wife..."
3,negative,Yesterday I took my husband to eat on his birt...
4,negative,Second and last time. Sadly I had to wait for ...


extract adjectives from text

In [5]:
# do word tokenization
clean_text = [[1]]*len(review)
for i in range(len(review)):
    clean_text[i]=clean_review(review['text'][i])

In [6]:
# get adjectives from all words
adj_words = [[1]]*len(review)
for i in range(len(review)):
    word=clean_text[i]
    pos_tag = nltk.pos_tag(word)
    adjs = []
    for words, pos in pos_tag:
        if (pos == 'JJ'):
            adjs.append(words)
    adj_words[i]=adjs

count frequencies for adjectives

In [7]:
# get all adjectives in one list
alladjwords = []
for i in range(len(review)):
    alladjwords=alladjwords + adj_words[i]
len(alladjwords)

1325513

In [8]:
# count the frequency for each adjectives
frequency = collections.Counter(alladjwords)
dict_frequency = dict(frequency)
print('the total number of adjectives in all reviews is',len(dict_frequency))

the total number of adjectives in all reviews is 29050


In [11]:
# get 1200 most common adjectives and then remove some wrong words
removeword=['u','dish','table','shrimp','noodle','sushi','salad','give','seafood','oyster','waitress','bread']
common = frequency.most_common(1200)
common=dict(common)
for word in removeword:
    common.pop(word)
print(common.keys())

dict_keys(['good', 'great', 'delicious', 'fresh', 'fish', 'nice', 'little', 'much', 'special', 'small', 'first', 'hot', 'new', 'next', 'favorite', 'bad', 'many', 'last', 'excellent', 'big', 'chinese', 'happy', 'sure', 'different', 'friendly', 'overall', 'full', 'large', 'worth', 'busy', 'awesome', 'huge', 'right', 'tasty', 'sweet', 'red', 'vega', 'restaurant', 'top', 'high', 'cheese', 'attentive', 'green', 'perfect', 'whole', 'fantastic', 'clean', 'long', 'open', 'garlic', 'italian', 'ok', 'amazing', 'mexican', 'second', 'extra', 'authentic', 'wonderful', 'old', 'super', 'hard', 'main', 'several', 'wrong', 'disappointed', 'fifteen', 'expensive', 'decent', 'chicken', 'entree', 'quick', 'free', 'white', 'fried', 'regular', 'meal', 'real', 'flavorful', 'japanese', 'reasonable', 'enough', 'cold', 'crawfish', 'lunch', 'ten', 'soft', 'slow', 'able', 'come', 'wait', 'dry', 'ordered', 'vegetable', 'black', 'wish', 'not_a', 'local', 'live', 'horrible', 'empty', 'entire', 'raw', 'fine', 'warm', 

get frequency matrix (row is the index of reviews and column is each adjective)

In [12]:
# get frequency matrix for adjectives
corpus=[]
corpus=list(review['text'])
df = pd.DataFrame(data=corpus, columns=['sentences'])
vectorizer = CountVectorizer(vocabulary=list(common.keys()), min_df=0)
X = vectorizer.fit_transform(df['sentences'].values)
result = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names())
result.head()

,good,great,delicious,fresh,fish,nice,little,much,special,small,...,hush,crowded,pull,tempe,inconsistent,gnocchi,not_up,definite,dip,forth
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


use multinomial Naive Bayes model to classify adjectives as positive or negative

In [13]:
# fit multinomial Naive Bayes model
Y=review['stars']
model = MultinomialNB()
model.fit(result, Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
# define classifier function and give two examples
def classifier(adjective):
    return model.predict(vectorizer.transform([adjective]))
print(classifier('great'))
print(classifier('bad'))

['positive']
['negative']


get and save dictionary for adjectives with positive or negative tags

In [15]:
# get dictionary
dict_adj={}
for word in common.keys():
    dict_adj[word]=classifier(word)[0]

In [16]:
# save dictionary
f = open('dict_adj.txt','w')
f.write(str(dict_adj))
f.close()